In [ ]:
import os
import numpy as np
import pandas as pd
import pickle
import lmdb
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
from torch_geometric.data import Data

In [ ]:
df=pd.read_json('zinc_ev.json')
df.head()

In [ ]:
df.shape

In [ ]:
df_shuffled = df.sample(frac=1)
n_train = 31601
n_val = 3999

In [ ]:
df_shuffled['E'][:n_train].mean()

In [ ]:
df_shuffled['E'][:n_train].std()

In [ ]:
db = lmdb.open('train.lmdb',
    map_size=1099511627776 * 2,
    subdir=False,
    meminit=False,
    map_async=True,
)

In [ ]:
i=0
for index, entry in tqdm(df_shuffled[:n_train].iterrows(), total=len(df_shuffled[:n_train])):
    
    atomic_numbers = torch.tensor(entry.atomic_number)
    pos = torch.tensor(entry.pos, dtype=torch.float32)
    natoms = torch.tensor(entry.natoms)
    fixed = torch.zeros(natoms, dtype=torch.float32)
    sid = entry.ID
    y = torch.tensor(entry.E)
    
    data = Data(
            pos=pos,
            atomic_numbers=atomic_numbers,
            natoms=natoms,
            sid=sid,
            fixed=fixed,
            y = y
        )
    
    txn = db.begin(write=True)
    txn.put(f"{i}".encode("ascii"), pickle.dumps(data, protocol=-1))
    txn.commit()
    db.sync()
    i+=1

db.close()  

In [ ]:
db = lmdb.open(
    'val.lmdb',
    map_size=1099511627776 * 2,
    subdir=False,
    meminit=False,
    map_async=True,
)


i=0
for index, entry in tqdm(df_shuffled[n_train:n_train+n_val].iterrows(), total=len(df_shuffled[n_train:n_train+n_val])):
    
    atomic_numbers = torch.tensor(entry.atomic_number)
    pos = torch.tensor(entry.pos, dtype=torch.float32)
    natoms = torch.tensor(entry.natoms)
    fixed = torch.zeros(natoms, dtype=torch.float32)
    sid = entry.ID
    y = torch.tensor(entry.E)
    
    data = Data(
            pos=pos,
            atomic_numbers=atomic_numbers,
            natoms=natoms,
            sid=sid,
            fixed=fixed,
            y = y
        )
    
    txn = db.begin(write=True)
    txn.put(f"{i}".encode("ascii"), pickle.dumps(data, protocol=-1))
    txn.commit()
    db.sync()
    i+=1

db.close()

In [ ]:
db = lmdb.open(
    'test.lmdb',
    map_size=1099511627776 * 2,
    subdir=False,
    meminit=False,
    map_async=True,
)


i=0
for index, entry in tqdm(df_shuffled[n_train+n_val:].iterrows(), total=len(df_shuffled[n_train+n_val:])):
    
    atomic_numbers = torch.tensor(entry.atomic_number)
    pos = torch.tensor(entry.pos, dtype=torch.float32)
    natoms = torch.tensor(entry.natoms)
    fixed = torch.zeros(natoms, dtype=torch.float32)
    sid = entry.ID
    y = torch.tensor(entry.E)
    
    data = Data(
            pos=pos,
            atomic_numbers=atomic_numbers,
            natoms=natoms,
            sid=sid,
            fixed=fixed,
            y = y
        )
    
    txn = db.begin(write=True)
    txn.put(f"{i}".encode("ascii"), pickle.dumps(data, protocol=-1))
    txn.commit()
    db.sync()
    i+=1

db.close()